In [2]:
import regression
import numpy as np

In [7]:
def loadDataSet(fileName):
    dataMat = []
    labelMat = []
    for line in open(fileName).readlines():
        dataList = [float(data) for data in line.strip().split('\t')]
        dataMat.append(dataList[0:-1])
        labelMat.append(dataList[-1])
    return dataMat, labelMat

In [20]:
def standRegres(xArr,yArr):
    X, y = np.array(xArr), np.array(yArr)
    xTx = X.T.dot(X)
    if np.linalg.det(xTx) == 0.0:
        print("This matrix is singular, cannot do inverse")
        return
    ws = np.mat(xTx).I.dot(X.T).dot(y)
    return ws

In [21]:
xArr,yArr=loadDataSet('ex0.txt')

In [22]:
xArr[0:2]

[[1.0, 0.067732], [1.0, 0.42781]]

In [23]:
ws = standRegres(xArr,yArr)

In [24]:
ws

matrix([[3.00774324, 1.69532264]])